# **Installing packages and importing libraries**

In [ ]:

!pip install -q pyomo
from pyomo.environ import *
import numpy as np
import pandas as pd

!apt-get install -y -qq glpk-utils

!apt-get install -y -qq coinor-cbc


In [ ]:
%%bash
rm -fv ipopt*
apt install -q -yy libgfortran5
sleep 1
wget -q https://www.ieor.iitb.ac.in/files/faculty/amahajan/ipopt
wget -q https://www.ieor.iitb.ac.in/files/faculty/amahajan/test.nl
chmod u+rx ipopt

!./ipopt test.nl

removed 'ipopt'
Reading package lists...
Building dependency tree...
Reading state information...
libgfortran5 is already the newest version (8.4.0-1ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.




bash: line 8: !./ipopt: No such file or directory


# EX2 : Part 1,2,3 : Problem formulation , model and solver

Lets take the house as the origion with coordinates (0.0). And the west denotes the positive x side, east denotes the negative x side and north and south be positive and negative y respectively.

Then our problem reduces to solving the following :


 \begin{array}{l}
Let\ ( x_{f} ,y_{f}) \ be\ the\ end\ point\ of\ the\ pipeline\ in\ the\ farm.\\
Let\ ( x_{l} ,y_{l}) \ be\ the\ start\ point\ of\ the\ pipeline\ in\ the\ lake.\\
\\
The\ shortest\ path\ between\ two\ points\ is\ say\ Z=\ \sqrt{( y_{f} -y_{l})^{2} +( x_{f} -x_{l})^{2}}\\
\\
Min\ \ Z=\ \ \sqrt{\ ( y_{f} -y_{l})^{2} +( x_{f} -x_{l})^{2}}\\
\\
s.t\\
\\
x_{f} +y_{f} \ \leqslant 0\\
x_{f} -y_{f} -200\ \leqslant \ 0\\
-x_{f} -y_{f} -200\ \leqslant 0\\
3y_{f} -x_{f} \ \leqslant \ 0\\
( x_{l} +200)^{2} +( y_{l} -200)^{2} \ \leqslant 50^{2}
\end{array}




In [ ]:
model = ConcreteModel()

model.xf = Var()
model.yf = Var()
model.xl = Var()
model.yl = Var()

model.obj = Objective(expr = (model.yf - model.yl) **2 +  (model.xf - model.xl) ** 2 , sense=minimize )
model.cons = ConstraintList()

model.cons.add(model.xf + model.yf <= 0)
model.cons.add(model.xf - model.yf - 200 <= 0)
model.cons.add((-1*model.xf) - model.yf - 200 <= 0)
model.cons.add(-1*model.xf + 3*model.yf <= 0)
model.cons.add( ((model.xl + 200)**2) +  ((model.yl-200)**2) - 2500  <= 0)

model.pprint()

solver = SolverFactory('ipopt')
result = solver.solve(model)
#print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('\nObjective : Z = ', sqrt(model.obj()))

print('\nDecision Variables')
print('(xf, yf)  = ','(', model.xf.value, ',', model.yf.value, ')')  
print('(xl, yl)  = ','(', model.xl.value, ',', model.yl.value, ')')




1 Set Declarations
    cons_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}

4 Var Declarations
    xf : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    xl : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    yf : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    yl : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : (yf - yl)**2 + (xf - xl)**2

1 Constraint Decla

# EX2 : Part 4,5,6 : Problem formulation , model and solver

Let (xj,yj) be the coordinates of the place where junction is and (xh, yh) be the location where the pipe from the junction ends in the store house.

 \begin{array}{l}
Squared\ Distance\ between\ lake\ and\ junction,\ Z_{1} \ =\ ( x_{j} -x_{l})^{2} +( y_{j} -y_{l})^{2}\\
Squared\ Distance\ between\ junction\ and\ the\ store\ house,\ Z_{2} \ =( x_{j} -x_{h})^{2} +( y_{j} -y_{h})^{2} \ \\
Squared\ Distance\ between\ junction\ and\ the\ farm,\ Z_{3} \ =\ ( x_{j} -x_{f})^{2} +( y_{j} -y_{f})^{2}\\
\\
\\
min\ :\ Z_{1} \ +\ 0.2Z_{2} \ +\ Z_{3}\\
s.t\\
\\
x_{f} +y_{f} \ \leqslant 0\\
x_{f} -y_{f} -200\ \leqslant \ 0\\
-x_{f} -y_{f} -200\ \leqslant 0\\
3y_{f} -x_{f} \ \leqslant \ 0\\
( x_{l} +200)^{2} +( y_{l} -200)^{2} \ \leqslant 50^{2}\\
\\
x_{h} \ =\ -15\\
y_{h} \ =\ 20
\end{array}


In [ ]:

model.xj = Var()
model.yj = Var()

model.xh = Var()
model.yh = Var()

model.obj = Objective(expr = ((model.yj - model.yl) **2 +  (model.xj - model.xl) ** 2) + 0.2*((model.yj - model.yh) **2 +  (model.xj - model.xh) ** 2) + ((model.yj - model.yf) **2 +  (model.xj - model.xf) ** 2)    , sense=minimize)

model.cons.add(model.xh == -15)
model.cons.add(model.yh == 20)

model.pprint()

solver = SolverFactory('ipopt')
result = solver.solve(model)
#print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('\nObjective :  = ', model.obj())

print('\nDecision Variables')
print('(xf, yf)  = ','(', model.xf.value, ',', model.yf.value, ')')  
print('(xl, yl)  = ','(', model.xl.value, ',', model.yl.value, ')')
print('(xj, yj)  = ','(', model.xj.value, ',', model.yj.value, ')')
print('(xh, yh)  = ','(', model.xh.value, ',', model.yh.value, ')')


    'pyomo.core.base.objective.SimpleObjective'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.objective.SimpleObjective'>). This
    is usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
1 Set Declarations
    cons_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    7 : {1, 2, 3, 4, 5, 6, 7}

8 Var Declarations
    xf : Size=1, Index=None
        Key  : Lower : Value               : Upper : Fixed : Stale : Domain
        None :  None : -120.00000000096625 :  None : False : False :  Reals
    xh : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    xj : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    xl : Size=1, Index=None
 